<a href="https://colab.research.google.com/github/beejhay31/100-days-of-code/blob/master/Unscaled_Hamoye_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')

In [3]:
column_names = {'T1':'Temp_Kitch(Cel)', 'RH_1': 'Humid_kitch(%)',
'T2': 'Temp_liv_r(Cel)', 'RH_2': 'Humid_liv_r(%)', 'T3': 'Temp_laund_r(Cel)', 'RH_3': 'Humid_laund_r(%)',
'T4': 'Temp_office_r(Cel)', 'RH_4': 'Humid_office_r(%)', 'T5': 'Temp_bath_r(Cel)', 'RH_5': 'Humid_bath_r(%)', 
'T6': 'Temp_out_build_North(Cel)', 'RH_6': 'Humid_out_build_North(%)',
'T7': 'Temp_iron_r(Cel)', 'RH_7': 'Humid_iron_r(%)', 'T8': 'Temp_teen_r(Cel)', 'RH_8': 'Humid_teen_r(%)',
'T9': 'Humid_par_room(Cel)', 'RH_9': 'Humid_par_room(%)',
'rv1': 'rand_var1', 'rv2': 'rand_var2'}

In [4]:
df = df.rename(columns=column_names)

In [5]:
df = df.drop(columns=['date', 'lights'])
target_var = df['Appliances']

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df, target_var, test_size=0.3, random_state=42)

In [7]:
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
linear_reg.fit(x_train, y_train)
predicted_values = linear_reg.predict(x_test)

In [8]:
pred_val_df=pd.DataFrame(predicted_values)
pred_val_df

,0
0,40.0
1,90.0
2,50.0
3,50.0
4,70.0
...,...
5916,250.0
5917,60.0
5918,50.0
5919,110.0


In [9]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 3) 

0.0

In [10]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 3)

1.0

In [11]:
import numpy as np
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 3)

0.0

In [12]:
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.0

In [13]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [14]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [15]:
from sklearn.linear_model import ElasticNet
e_net = ElasticNet(alpha = 0.001) 
e_net.fit(x_train, y_train) 

ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [16]:
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

In [17]:
linear_model_weights = get_weights_df(linear_reg, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
e_weights_df = get_weights_df(e_net, x_train, 'E_weights')

In [18]:
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights=pd.merge(final_weights, e_weights_df, on='Features')

In [19]:
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight,E_weights
0,Temp_iron_r(Cel),-7.087772e-15,4.147740e-09,0.0,0.0
1,Temp_Kitch(Cel),-5.476270e-15,-1.484659e-09,0.0,0.0
2,Humid_liv_r(%),-2.950576e-15,-5.484821e-08,-0.0,-0.0
3,Humid_teen_r(%),-2.326982e-15,-2.307086e-08,-0.0,-0.0
4,Temp_bath_r(Cel),-2.035198e-15,-6.397142e-09,0.0,0.0
5,Temp_out_build_North(Cel),-1.461417e-15,2.939252e-08,0.0,0.0
6,Humid_office_r(%),-1.166994e-15,4.808971e-09,0.0,0.0
7,Temp_liv_r(Cel),-7.620131e-16,-7.331837e-08,0.0,0.0
8,Visibility,-4.427981e-16,8.087579e-10,0.0,0.0
9,Press_mm_hg,-2.091080e-16,6.795164e-10,-0.0,-0.0


In [20]:
 from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.0